# XGBoost Model - Energy Cost Prediction with Time Series

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

## Import Data as Dataframe

- Import our cleaned data
- Set datetime column as index
- Convert our index to datetime object

In [ ]:
df = pd.read_csv('./data/prepared/df_energy_climate_2020.csv')
df = df.set_index('datetime')
df.index = pd.to_datetime(df.index)

## Visualize our data - Energy Price Transition in 2020

In [ ]:
df['energy_price'].plot(
    style='.', 
    figsize=(15, 5),
    color=color_pal[0], # type: ignore
    title='Energy price in 2020'
)

## Train / Test Split

- Split the data into Train : Test = 8 : 2

In [ ]:
splitting_point = (int(len(df)*0.2))
train, test = df[:-splitting_point], df[-splitting_point:]

## Visualize Train / Test Data

In [ ]:
fig, ax = plt.subplots(figsize =(15, 5))
train.plot(ax=ax, y='energy_price', label = 'Training Set', title='Data Train/Test Split')
test.plot(ax=ax, y='energy_price', label = 'Testing Set')
ax.axvline('2020-10-19', color='black', ls='--') # type: ignore
plt.show()

In [ ]:
df.index[-splitting_point]

## Visualize Data of one week

In [ ]:
df.loc[(df.index > '2020-11-01') & (df.index < '2020-11-08')]['energy_price'].plot(figsize=(15,5), title='Week of data')

## Set our Feature and Target
- Target = Energy price
- Features = Hour, Day of Week, Month

In [ ]:
FEATURES = ['hour', 'dayofweek', 'month']
TARGET = 'energy_price'

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

## Create our Model

In [ ]:
# Set random state to ensure reproductivity
reg = xgb.XGBRegressor(random_state=0)

## Train our Model and make Predictions

In [ ]:
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100
)

## Visualize Truth and Predictions

In [ ]:
test['prediction'] = reg.predict(X_test)
df = df.merge(test[['prediction']], how='left', left_index=True, right_index=True)
ax = df[['energy_price']].plot(figsize=(15, 5))
df['prediction'].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Predictions'])
ax.set_title('Raw Data and Prediction')
plt.show()

In [ ]:
ax = df.loc[(df.index > '2020-11-01') & (df.index < '2020-11-08')]['energy_price'].plot(figsize=(15,5), title='Week of data')
df.loc[(df.index > '2020-11-01') & (df.index < '2020-11-08')]['prediction'].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Prediction'])
plt.show()

## Evaluate our Model
- We use two evaluation metrics
    - RMSE 
    - MAPE

- RMSE (Root Mean Squared Error)
    - Measures the average distance between the predicted values and the actual values (squared difference is taking into account)
    - RMSE gives more weight to **larger errors** and is useful when it is important to minimize the impact of large errors
    - It can be heavily influenced by outliers
    - Useful when comparing the performance of different models
    - It is not always easy to interpret the magnitude of the error as it is not a percentage
- MAPE (Mean Absolute Percentae Error)
    - Measures the average percentage difference between the predicted values and actual values
    - Useful when the data contains different scales or magnitudes, as it normalizes the error by the actual value
    - It is more interpretable
    - It may be undefined or produce very large values when the actual values are close to zero
    - It is not symmetric => The errors in the predicted values may not be weighted equally in both directions

In [ ]:
score_rmse = np.sqrt(mean_squared_error(test['energy_price'], test['prediction']))
print(f'RMSE Score on test set: {score_rmse:.2f}')
score_mape = (mean_absolute_percentage_error(test['energy_price'], test['prediction']))
print(f'MAPE Score on test set: {score_mape:.2f}')

| Model | Characteristics of the model       | RMSE | MAPE |
| --| -- | :----------------------: | :----------------------: | 
| Model 1 | XGBoost Regressor with Features Month, Day of week and Hour| 16.66 | 11.70% |           

## Search for hyperparameters

### What are hyperparameters?
- They re not lerned by the model during the training but set beforehand by the user or the researcher
- They can have a significant impact on the performance of the model 

In [ ]:
reg_hyper_p = xgb.XGBRegressor(random_state=0)

- We use grid search
- We search for the best parameters for 
    - n_estimators: the number of trees built before taking the maximum voting or averages of predictions
    - max_depth: the longest path between the root node and the leaf node 
    - gamma: controls minimum loss reduction required to split a node during tree construction
    - learning_rate: used to govern the pace at which an algorithm updates or learns the values of a parameter estimate

In [ ]:
# make a dictionary of hyperparameter values to search
search_space = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 6, 9],
    'gamma': [0.01, 0.1],
    'learning_rate': [0.001, 0.01, 0.1],
}

GS = GridSearchCV(estimator = reg_hyper_p,
                  param_grid = search_space,
                  scoring=['r2', 'neg_root_mean_squared_error'],
                  refit = 'r2', # type: ignore
                  cv = 5,
                #   verbose=4
)

In [16]:
GS.fit(X_train, y_train)

In [ ]:
print(GS.best_params_)

## Create a Model with Hyperparameters

In [ ]:
reg_hyper_p = xgb.XGBRegressor(
    gamma=0.1,
    n_estimators=500,
    max_depth=3,
    learning_rate=0.01,
    random_state=0
)

In [ ]:
reg_hyper_p.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100
)

In [ ]:
test['prediction_hyp'] = reg_hyper_p.predict(X_test)
df = df.merge(test[['prediction_hyp']], how='left', left_index=True, right_index=True)
ax = df[['energy_price']].plot(figsize=(15, 5))
df['prediction_hyp'].plot(ax=ax, style='.')
plt.legend(['Trueth Data', 'Predictions'])
ax.set_title('Raw Data and Prediction')
plt.show()

In [ ]:
ax = df.loc[(df.index > '2020-11-01') & (df.index < '2020-11-08')]['energy_price'].plot(figsize=(15,5), title='Week of data')
df.loc[(df.index > '2020-11-01') & (df.index < '2020-11-08')]['prediction_hyp'].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Prediction with Hyperparameters'])
plt.show()

In [ ]:
score_rmse = np.sqrt(mean_squared_error(test['energy_price'], test['prediction_hyp']))
print(f'RMSE Score on test set: {score_rmse:.2f}')
score_mape = (mean_absolute_percentage_error(test['energy_price'], test['prediction_hyp']))
print(f'MAPE Score on test set: {score_mape:.2f}')

| Model | Characteristics of the model       | RMSE | MAPE |
| --| -- | :----------------------: | :----------------------: | 
| Model 1 | XGBoost Regressor with Features Month, Day of week and Hour| 16.66 | 11.70% |    
| Model 2 | Model 1 with hyperparameters|15.89 | 12.59% |          


### RMSE improved slightly, but MAPE worsened. What does it mean?
This may occur when the model is accurately predicting the values close to the actual values, but making larger errors for the values that are farther from the actual values.

## Removing outliers

In [ ]:
df['energy_price'].plot(kind='hist', bins=500)

In [ ]:
df.query('energy_price > 100').plot(y='energy_price', figsize=(15, 5), style='.')

In [ ]:
df.query('energy_price < -20').plot(y='energy_price', figsize=(15, 5), style='.')

In [ ]:
df = df.query('energy_price < 100').copy()
df = df.query('energy_price > -20').copy()

In [ ]:
splitting_point = (int(len(df)*0.2))
train, test = df[:-splitting_point], df[-splitting_point:]
X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

reg_hyper_p.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=100
)

In [ ]:
test['prediction_less_outliers'] = reg_hyper_p.predict(X_test)
df = df.merge(test[['prediction_less_outliers']], how='left', left_index=True, right_index=True)
ax = df[['energy_price']].plot(figsize=(15, 5))
df['prediction_less_outliers'].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Predictions'])
ax.set_title('Raw Data and Prediction')
plt.show()

In [ ]:
score_rmse = np.sqrt(mean_squared_error(test['energy_price'], test['prediction_less_outliers']))
print(f'RMSE Score on test set: {score_rmse:.2f}')
score_mape = (mean_absolute_percentage_error(test['energy_price'], test['prediction_less_outliers']))
print(f'MAPE Score on test set: {score_mape:.2f}')

| Model | Characteristics of the model       | RMSE | MAPE |
| --| -- | :----------------------: | :----------------------: | 
| Model 1 | XGBoost Regressor with Features Month, Day of week and Hour| 16.66 | 11.70% |    
| Model 2 | Model 1 with hyperparameters|15.89 | 12.59% |               
| Model 3 | Model 2 with data from which outliers are removed |14.24  |12.76%  | 

## Time Series Cross Validation

### What is Cross Validation?
Cross-validation is an essential technique used in machine learning to evaluate the performance of a model on unseen data.

### Benefits of Cross Validation
- It provides a more accurate estimate of a model's performance on new data
- Helps to identify overfitting
- Allows for tuning of hyperparameters to optimize model performance

In [ ]:
tss = TimeSeriesSplit(n_splits=5, test_size=168, gap=168)
df = df.sort_index()

fig, axs = plt.subplots(5, 1, figsize=(15, 8), sharex=True)

fold = 0
preds = []
scores_rmse = []
scores_mape = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]
    train['energy_price'].plot(ax=axs[fold],
                               label='Training Set',
                               title=f'Data Train/Test Split Fold {fold}')
    test['energy_price'].plot(ax=axs[fold],
                              label='Test Set')
    axs[fold].axvline(test.index.min(), color='black', ls='--')

    FEATURES = ['month', 'hour', 'dayofweek']
    TARGET = 'energy_price'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(
        gamma=0.1,
        n_estimators=500,
        max_depth=3,
        learning_rate=0.01,
        random_state=0
    )
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)
    
    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    scores_rmse.append(score_rmse)
    score_mape = (mean_absolute_percentage_error(y_test, y_pred))
    scores_mape.append(score_mape)
    
    fold += 1

In [ ]:
print(f'RMSE scores across folds {np.mean(scores_rmse):.2f}')
print(f'Fold RMSE scores: {scores_rmse}')
print(f'MAPE scores across folds {np.mean(scores_mape):.2f}')
print(f'Fold MAPE scores: {scores_mape}')

| Model | Characteristics of the model       | RMSE | MAPE |
| --| -- | :----------------------: | :----------------------: | 
| Model 1 | XGBoost Regressor with Features Month, Day of week and Hour| 16.66 | 11.70% |    
| Model 2 | Model 1 with hyperparameters|15.89 | 12.59% |               
| Model 3 | Model 2 with data from which outliers are removed |14.71  |12.66%  | 
| Model 3 | Results in Cross Validation |18.49  | 12.60% | 

## Time Series Cross Validation with Lag Features

In [ ]:
def add_past_covariants(df, column):
    df[f'{column}_lag1'] = df[column].shift(168)
    df[f'{column}_lag2'] = df[column].shift(24)
    df[f'{column}_lag3'] = df[column].shift(1)
    
    return df

In [ ]:
df = add_past_covariants(df, 'energy_price')
df = add_past_covariants(df, 'wind_speed')
df = add_past_covariants(df, 'solar_radiation')
df = add_past_covariants(df, 'renewable')
df = add_past_covariants(df, 'not_renewable')
df = add_past_covariants(df, 'nuclear_power')

## Take All Past Covariates

In [ ]:
tss = TimeSeriesSplit(n_splits=5, test_size=168, gap=168)
df = df.sort_index()

fold = 0
preds = []
scores_rmse = []
scores_mape = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    FEATURES = ['month', 'hour', 'dayofweek', 'energy_price_lag1', 'energy_price_lag2', 'energy_price_lag3', 'wind_speed_lag1', 'wind_speed_lag2', 'wind_speed_lag3', 'solar_radiation_lag1', 'solar_radiation_lag2', 'solar_radiation_lag3', 'renewable_lag1', 'renewable_lag2', 'renewable_lag3', 'not_renewable_lag1', 'not_renewable_lag2', 'not_renewable_lag3', 'nuclear_power_lag1', 'nuclear_power_lag2', 'nuclear_power_lag3']
    TARGET = 'energy_price'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(
        gamma=0.1,
        n_estimators=500,
        max_depth=3,
        learning_rate=0.01,
        random_state=0
    )
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)
    
    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    scores_rmse.append(score_rmse)
    score_mape = (mean_absolute_percentage_error(y_test, y_pred))
    scores_mape.append(score_mape)
    fold += 1

In [ ]:
print(f'RMSE scores across folds {np.mean(scores_rmse):.2f}')
print(f'Fold RMSE scores: {scores_rmse}')
print(f'MAPE scores across folds {np.mean(scores_mape):.2f}')
print(f'Fold MAPE scores: {scores_mape}')

| Model | Characteristics of the model       | RMSE | MAPE |
| --| -- | :----------------------: | :----------------------: | 
| Model 1 | XGBoost Regressor with Features Month, Day of week and Hour| 16.66 | 11.70% |    
| Model 2 | Model 1 with hyperparameters|15.89 | 12.59% |               
| Model 3 | Model 2 with data from which outliers are removed |14.71  |12.66%  | 
| Model 3 | Results in Cross Validation |16.76  | 11.91% | 
| Model 4 | Model 3 with lag features(1), results in Cross validation | 5.59  | 1.67%  | 

(1) Below features lagged by one week, one day and one hour are included:
- Energy price
- Wind speed
- Solar radiation
- Renewable energy feeding volume
- Not renewable energy feeding volume
- Nuclear energy feeding volume

## Feature Importance

In [ ]:
fi = pd.DataFrame(
    data=reg.feature_importances_,
    index=reg.feature_names_in_,
    columns=['importance']
)
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

## Remove Past Covariates with less Importance

In [ ]:
# gap: Number of samples to exclude from the end of each train set before the test set.
tss = TimeSeriesSplit(n_splits=5, test_size=168, gap=168)
df = df.sort_index()

fold = 0
preds = []
scores_rmse = []
scores_mape = []
for train_idx, val_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[val_idx]

    FEATURES = ['hour', 'dayofweek', 'energy_price_lag1', 'energy_price_lag2', 'energy_price_lag3', 'solar_radiation_lag3', 'renewable_lag3', 'not_renewable_lag3']
    TARGET = 'energy_price'

    X_train = train[FEATURES]
    y_train = train[TARGET]

    X_test = test[FEATURES]
    y_test = test[TARGET]

    reg = xgb.XGBRegressor(
        gamma=0.1,
        n_estimators=500,
        max_depth=3,
        learning_rate=0.01,
        random_state=0
    )
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            verbose=100)
    
    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    scores_rmse.append(score_rmse)
    score_mape = (mean_absolute_percentage_error(y_test, y_pred))
    scores_mape.append(score_mape)
    fold += 1

In [ ]:
print(f'RMSE scores across folds {np.mean(scores_rmse):.2f}')
print(f'Fold RMSE scores: {scores_rmse}')
print(f'MAPE scores across folds {np.mean(scores_mape):.2f}')
print(f'Fold MAPE scores: {scores_mape}')

| Model | Characteristics of the model       | RMSE | MAPE |
| --| -- | :----------------------: | :----------------------: | 
| Model 1 | XGBoost Regressor with Features Month, Day of week and Hour| 16.66 | 11.70% |    
| Model 2 | Model 1 with hyperparameters|15.89 | 12.59% |               
| Model 3 | Model 2 with data from which outliers are removed |14.71  |12.66%  | 
| Model 3 | Results in Cross Validation |16.76  | 11.91% | 
| Model 4 | Model 3 with lag features(1), results in Cross validation | 5.71  | 1.71%  | 
| Model 5 | Model 3 with selected lag features(2) in Cross validation | 5.51  | 1.68%  | 

(1) Below features lagged by one week, one day and one hour are included:
- Energy price
- Wind speed
- Solar radiation
- Renewable energy feeding volume
- Not renewable energy feeding volume
- Nuclear energy feeding volume

(2) Below features are included:
- Energy price lagged by one week
- Energy price lagged by one day
- Energy price lagged by one hour
- Solar radiation lagged by one hour
- Renewable energy feeding volume lagged by one hour
- Not renewable energy feeding volume lagged by one hour

## Finalize and Evaluate

In [ ]:
# Import dataframe
df = pd.read_csv('./data/prepared/df_energy_climate_2020.csv')
df = df.set_index('datetime')
df.index = pd.to_datetime(df.index)

# Remove outliers
df = df.query('energy_price < 105').copy()
df = df.query('energy_price > -50').copy()

one_week = 168

# Add past covariants
def add_past_covariants(df, column):
    df[f'{column}_cov'] = df[column]
    df.loc[(df.index >= df.index[-one_week], f'{column}_cov' )] = np.nan
    df[f'{column}_lag1'] = df[column].shift(168)
    df[f'{column}_lag2'] = df[column].shift(24)
    df[f'{column}_lag3'] = df[column].shift(1)
    
    return df

df = add_past_covariants(df, 'energy_price')
df = add_past_covariants(df, 'wind_speed')
df = add_past_covariants(df, 'solar_radiation')
df = add_past_covariants(df, 'renewable')
df = add_past_covariants(df, 'not_renewable')
df = add_past_covariants(df, 'nuclear_power')

# Train / Test split
# We use the last week of the data as test data
train, test = df[:-one_week], df[-one_week:]

# Add features and set target
FEATURES = ['hour', 'dayofweek', 'energy_price_lag1', 'energy_price_lag2', 'energy_price_lag3', 'solar_radiation_lag3', 'renewable_lag3', 'not_renewable_lag3']
TARGET = 'energy_price'

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# Create a model
reg = xgb.XGBRegressor(
    gamma=0.1,
    n_estimators=500,
    max_depth=3,
    learning_rate=0.01,
    random_state=0
)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)

# Make prediction
y_pred = reg.predict(X_test)

# Evaluate the model
score_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
score_mape = (mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
test['y_pred'] = y_pred
df = df.merge(test[['y_pred']], how='left', left_index=True, right_index=True)
ax = df[['energy_price']].plot(figsize=(15, 5))
df['y_pred'].plot(ax=ax, style='.')
plt.legend(['Truth Data', 'Predictions'])
ax.set_title('Raw Data and Prediction')
plt.show()

In [ ]:
score_rmse = np.sqrt(mean_squared_error(test['energy_price'], test['y_pred']))
print(f'RMSE Score on test set: {score_rmse:.2f}')
score_mape = (mean_absolute_percentage_error(test['energy_price'], test['y_pred']))
print(f'MAPE Score on test set: {score_mape:.2f}')

| Model | Characteristics of the model       | RMSE | MAPE |
| --| -- | :----------------------: | :----------------------: | 
| Model 1 | XGBoost Regressor with Features Month, Day of week and Hour| 16.66 | 11.70% |    
| Model 2 | Model 1 with hyperparameters|15.89 | 12.59% |               
| Model 3 | Model 2 with data from which outliers are removed |14.71  |12.66%  | 
| Model 3 | Results in Cross Validation |16.76  | 11.91% | 
| Model 4 | Model 3 with lag features(1), results in Cross validation | 5.71  | 1.71%  | 
| Model 5 | Model 3 with selected lag features(2) in Cross validation | 5.59  | 1.74%  | 
| Model 5 | Model 5, train : test = 51 weeks : 1 week | 4.76  | 3.13%  | 

(1) Below features lagged by one week, one day and one hour are included:
- Energy price
- Wind speed
- Solar radiation
- Renewable energy feeding volume
- Not renewable energy feeding volume
- Nuclear energy feeding volume

(2) Below features are included:
- Energy price lagged by one week
- Energy price lagged by one day
- Energy price lagged by one hour
- Solar radiation lagged by one hour
- Renewable energy feeding volume lagged by one hour
- Not renewable energy feeding volume lagged by one hour

## Train Model with All Data and Predict Future

In [ ]:
# Import dataframe
df = pd.read_csv('./data/prepared/df_energy_climate_2020.csv')
df = df.set_index('datetime')
df.index = pd.to_datetime(df.index)

# Remove outliers
df = df.query('energy_price < 100').copy()
df = df.query('energy_price > -20').copy()

# Create a dataframe for future
future = pd.date_range('2021-01-01', '2021-01-08', freq='1h')
future_df = pd.DataFrame(index=future)
future_df.index = pd.to_datetime(future_df.index)
future_df['month'] = future_df.index.month
future_df['dayofweek'] = future_df.index.dayofweek
future_df['hour'] = future_df.index.hour

future_df['isFuture'] = True
df['isFuture'] = False

# Combine the data and future dataframe
df_and_future = pd.concat([df, future_df])

# Add past covariants
def add_past_covariants(df, column):
    df[f'{column}_lag1'] = df[column].shift(168)
    df[f'{column}_lag2'] = df[column].shift(24)
    df[f'{column}_lag3'] = df[column].shift(1)
    
    return df

df_and_future = add_past_covariants(df_and_future, 'energy_price')
df_and_future = add_past_covariants(df_and_future, 'wind_speed')
df_and_future = add_past_covariants(df_and_future, 'solar_radiation')
df_and_future = add_past_covariants(df_and_future, 'renewable')
df_and_future = add_past_covariants(df_and_future, 'not_renewable')
df_and_future = add_past_covariants(df_and_future, 'nuclear_power')

df_and_future.to_csv('./future.csv')

# Extract the part for future
future_w_features = df_and_future.query('isFuture').copy()


# Create a model
reg = xgb.XGBRegressor(
    gamma=0.1,
    n_estimators=500,
    max_depth=3,
    learning_rate=0.01,
    random_state=0
)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)

# Make prediction
future_w_features['pred'] = reg.predict(future_w_features[FEATURES])

future_w_features['pred'].plot(
    figsize=(10, 5),
    color='purple',
    ms=1,
    lw=1,
    title= 'Future Prediction'
)
